In [ ]:
import random
import numpy as np
from copulas.multivariate import Multivariate
import pandas as pd

In [ ]:
import trading_vix_objective_2
env = trading_vix_objective_2.trading_vix()

In [ ]:
env.reset()

In [ ]:
for time_step in range(0,10):
    action = random.uniform(0, 1)
    obs,reward = env.step(action)
    #print(obs)
    print(reward)
    print(' ')

In [ ]:
free_parameters = \
{'cov_para_1': -0.8419647954404355, 'cov_para_2': -0.785625296831131, 'cov_para_3': 0.28262189626693723, 'cov_para_4': -0.14224736094474788, 'cov_para_5': -0.3787086009979248, 'cov_para_6': -0.7235054165124893, 'beta_1a': 11.93993330001831, 'beta_1b': 13.292677402496338, 'beta_2a': 1.1339520663022995, 'beta_2b': 11.920332908630371, 'beta_3a': 17.002562284469604, 'beta_3b': 1.0662934929132462, 'beta_4a': 7.662838101387024, 'beta_4b': 6.5861910581588745, 'lambda_expon_1': 0.09136121791601182, 'lambda_expon_2': 0.0949619756937027, 'lambda_expon_3': 0.04050088679790497}

In [ ]:
free_para1 = free_parameters['cov_para_1']
free_para2 = free_parameters['cov_para_2']
free_para3 = free_parameters['cov_para_3']
free_para4 = free_parameters['cov_para_4']
free_para5 = free_parameters['cov_para_5']
free_para6 = free_parameters['cov_para_6']

diag_1 = 1
diag_2 = np.sqrt(1-free_para1**2)
if (1-free_para2**2-free_para3**2)<0:
    diag_3 = np.sqrt(np.abs(1-free_para2**2-free_para3**2))
else:
    diag_3 = np.sqrt(1-free_para2**2-free_para3**2)
if (1-free_para4**2-free_para5**2-free_para6**2)<0:
    diag_4 = np.sqrt(np.abs(1-free_para4**2-free_para5**2-free_para6**2))
else:
    diag_4 = np.sqrt(1-free_para4**2-free_para5**2-free_para6**2)


lower_triangular_matrix = np.asarray([[diag_1,0,0,0],
                        [free_para1,diag_2,0,0],
                        [free_para2,free_para3,diag_3,0],
                        [free_para4,free_para5,free_para6,diag_4]])


cov_matrix = lower_triangular_matrix@lower_triangular_matrix.transpose()


#now, define the marginal distribution of the gaussian copula
univerates = [{'loc': 0,
'scale': 1,
'a': free_parameters['beta_1a'],
'b': free_parameters['beta_1b'],
'type': 'copulas.univariate.beta.BetaUnivariate'},
{'loc': 0,
'scale': 1,
'a': free_parameters['beta_2a'],
'b': free_parameters['beta_2b'],
'type': 'copulas.univariate.beta.BetaUnivariate'},
{'loc': 0,
'scale': 1,
'a': free_parameters['beta_3a'],
'b': free_parameters['beta_3b'],
'type': 'copulas.univariate.beta.BetaUnivariate'},
{'loc': 0,
'scale': 1,
'a': free_parameters['beta_4a'],
'b': free_parameters['beta_4b'],
'type': 'copulas.univariate.beta.BetaUnivariate'}]


#now, we construct the gaussian copula
copula_parameters = {}
copula_parameters['covariance'] = cov_matrix
copula_parameters['univariates'] = univerates
copula_parameters['type'] = 'copulas.multivariate.gaussian.GaussianMultivariate'
copula_parameters['columns'] = [0,1,2,3]


new_dist = Multivariate.from_dict(copula_parameters)

In [ ]:
cov_matrix

In [ ]:
from numpy import linalg as LA
eig_value, eig_vectors = LA.eig(cov_matrix)
eig_value

In [ ]:
from scipy.stats import expon

In [ ]:
env = trading_vix_objective_2.trading_vix()
current_feature = env.reset()

In [ ]:
lambda_expon_1 = free_parameters['lambda_expon_1']
lambda_expon_2 = free_parameters['lambda_expon_2']
lambda_expon_3 = free_parameters['lambda_expon_3']
lambda_expons = [lambda_expon_1,lambda_expon_2,lambda_expon_3]


transformed_features = []
for feature_index in range(len(lambda_expons)):
    transformation = expon.cdf(current_feature[feature_index,0],scale = 1.0/lambda_expons[feature_index])
    transformed_features.append(transformation)
transformed_features = np.asarray(transformed_features)
transformed_features = np.reshape(transformed_features,(1,-1))
holding_position = np.reshape(current_feature[-1,:],(1,1))
holding_position = np.asarray([[0.10]])
data_point_for_df = np.concatenate((transformed_features,holding_position),axis = 1)

print(data_point_for_df)

data_point_for_copula = pd.DataFrame(data_point_for_df)


assert data_point_for_df.shape[1] == 4
action = new_dist.cdf(data_point_for_copula)
print(action)

In [ ]:
current_feature

In [ ]:
current_feature[-1,:]

In [ ]:
from scipy.special import expit

expit(current_feature[-1,:])


In [ ]:
free_parameters = \
{'cov_para_1': -0.8419647954404355, 'cov_para_2': -0.785625296831131, 'cov_para_3': 0.28262189626693723, 'cov_para_4': -0.14224736094474788, 'cov_para_5': -0.3787086009979248, 'cov_para_6': -0.7235054165124893, 'beta_1a': 11.93993330001831, 'beta_1b': 13.292677402496338, 'beta_2a': 1.1339520663022995, 'beta_2b': 11.920332908630371, 'beta_3a': 17.002562284469604, 'beta_3b': 1.0662934929132462, 'beta_4a': 7.662838101387024, 'beta_4b': 6.5861910581588745, 'lambda_expon_1': 0.09136121791601182, 'lambda_expon_2': 0.0949619756937027, 'lambda_expon_3': 0.04050088679790497}

In [ ]:
free_para1 = free_parameters['cov_para_1']
free_para2 = free_parameters['cov_para_2']
free_para3 = free_parameters['cov_para_3']
free_para4 = free_parameters['cov_para_4']
free_para5 = free_parameters['cov_para_5']
free_para6 = free_parameters['cov_para_6']

diag_1 = 1
diag_2 = np.sqrt(1-free_para1**2)
if (1-free_para2**2-free_para3**2)<0:
    diag_3 = np.sqrt(np.abs(1-free_para2**2-free_para3**2))
else:
    diag_3 = np.sqrt(1-free_para2**2-free_para3**2)
if (1-free_para4**2-free_para5**2-free_para6**2)<0:
    diag_4 = np.sqrt(np.abs(1-free_para4**2-free_para5**2-free_para6**2))
else:
    diag_4 = np.sqrt(1-free_para4**2-free_para5**2-free_para6**2)


lower_triangular_matrix = np.asarray([[diag_1,0,0,0],
                        [free_para1,diag_2,0,0],
                        [free_para2,free_para3,diag_3,0],
                        [free_para4,free_para5,free_para6,diag_4]])

lower_triangular_matrix = np.asarray([[diag_1,0,0],
                        [free_para1,diag_2,0],
                        [free_para2,free_para3,diag_3]])


cov_matrix = lower_triangular_matrix@lower_triangular_matrix.transpose()


#now, define the marginal distribution of the gaussian copula
univerates = [{'loc': 0,
'scale': 1,
'a': free_parameters['beta_1a'],
'b': free_parameters['beta_1b'],
'type': 'copulas.univariate.beta.BetaUnivariate'},
{'loc': 0,
'scale': 1,
'a': free_parameters['beta_2a'],
'b': free_parameters['beta_2b'],
'type': 'copulas.univariate.beta.BetaUnivariate'},
{'loc': 0,
'scale': 1,
'a': free_parameters['beta_3a'],
'b': free_parameters['beta_3b'],
'type': 'copulas.univariate.beta.BetaUnivariate'},
# {'loc': 0,
# 'scale': 1,
# 'a': free_parameters['beta_4a'],
# 'b': free_parameters['beta_4b'],
# 'type': 'copulas.univariate.beta.BetaUnivariate'}
]


#now, we construct the gaussian copula
copula_parameters = {}
copula_parameters['covariance'] = cov_matrix
copula_parameters['univariates'] = univerates
copula_parameters['type'] = 'copulas.multivariate.gaussian.GaussianMultivariate'
copula_parameters['columns'] = [0,1,2]


new_dist = Multivariate.from_dict(copula_parameters)

In [ ]:
lower_triangular_matrix

In [ ]:
from numpy import linalg as LA
eig_value, eig_vectors = LA.eig(cov_matrix)
eig_value

In [ ]:
data_points = pd.DataFrame()
data_points[0] = [0.9]
data_points[1] = [0.8]
data_points[2] = [0.9]
#data_points[3] = [0.0]
print(data_points)

In [ ]:
new_dist.cdf(data_points)

In [ ]:
def construct_a_copula(cov_input,marginal_input):
    
    assert len(cov_input)<4
        
    if len(cov_input)==3:
        diag_1 = 1
        diag_2 = np.sqrt(1-cov_input[0]**2)
        if (1-cov_input[1]**2-cov_input[2]**2)<0:
            diag_3 = np.sqrt(np.abs(1-cov_input[1]**2-cov_input[2]**2))
        else:
            diag_3 = np.sqrt(1-cov_input[1]**2-cov_input[2]**2)

        lower_triangular_matrix = np.asarray([[diag_1,0,0],
                                [cov_input[0],diag_2,0],
                                [cov_input[1],cov_input[2],diag_3]])

        cov_matrix = lower_triangular_matrix@lower_triangular_matrix.transpose()

    
    if len(cov_input)==1:
        diag_2 = np.sqrt(1-cov_input[0]**2)
        
        lower_triangular_matrix = np.asarray([[diag_1,0],
                                [cov_input[0],diag_2]])
        
        cov_matrix = lower_triangular_matrix@lower_triangular_matrix.transpose()
    
    
    if len(marginal_input)== 6:
        univerates = [{'loc': 0,
        'scale': 1,
        'a': marginal_input[0],
        'b': marginal_input[1],
        'type': 'copulas.univariate.beta.BetaUnivariate'},
        {'loc': 0,
        'scale': 1,
        'a': marginal_input[2],
        'b': marginal_input[3],
        'type': 'copulas.univariate.beta.BetaUnivariate'},
        {'loc': 0,
        'scale': 1,
        'a': marginal_input[4],
        'b': marginal_input[5],
        'type': 'copulas.univariate.beta.BetaUnivariate'}]
        
    if len(marginal_input)==4:
        univerates = [{'loc': 0,
        'scale': 1,
        'a': marginal_input[0],
        'b': marginal_input[1],
        'type': 'copulas.univariate.beta.BetaUnivariate'},
        {'loc': 0,
        'scale': 1,
        'a': marginal_input[2],
        'b': marginal_input[3],
        'type': 'copulas.univariate.beta.BetaUnivariate'}]
    
    copula_parameters = {}
    copula_parameters['covariance'] = cov_matrix
    copula_parameters['univariates'] = univerates
    copula_parameters['type'] = 'copulas.multivariate.gaussian.GaussianMultivariate'
    if len(marginal_input)== 6:
        copula_parameters['columns'] = [0,1,2]
    if len(marginal_input)==4:
        copula_parameters['columns'] = [0,1]
        
    new_dist = Multivariate.from_dict(copula_parameters)
    
    return new_dist

In [ ]:
free_para1 = free_parameters['cov_para_1']
free_para2 = free_parameters['cov_para_2']
free_para3 = free_parameters['cov_para_3']

In [ ]:
cov_input = [free_para1,free_para2,free_para3]
marginal_input = [free_parameters['beta_1a'],free_parameters['beta_1b'],
                 free_parameters['beta_2a'],free_parameters['beta_2b'],
                 free_parameters['beta_3a'],free_parameters['beta_3b']]

In [ ]:
dist = construct_a_copula(cov_input,marginal_input)

In [ ]:
data_point_for_copula

In [ ]:
new_dist.cdf(data_points)

In [ ]:
import trading_vix_objective_2
env = trading_vix_objective_2.trading_vix()

In [ ]:
env.reset()

In [ ]:
current_feature = env.reset()

In [ ]:
transformed_features = []
for feature_index in range(len(lambda_expons)):
    transformation = expon.cdf(current_feature[feature_index,0],scale = 1.0/lambda_expons[feature_index])
    transformed_features.append(transformation)
transformed_features = np.asarray(transformed_features)

In [ ]:
transformed_features

In [ ]:
transformed_features

In [ ]:
transformed_features = np.reshape(transformed_features,(1,-1))

In [ ]:
transformed_features.shape

In [ ]:
data_point_for_copula = pd.DataFrame(transformed_features)
data_point_for_copula

In [ ]:
current_feature[-1,:][0]

In [ ]:
transformed_features = []
for feature_index in range(len(lambda_expons)):
    transformation = expon.cdf(current_feature[feature_index,0],scale = 1.0/lambda_expons[feature_index])
    transformed_features.append(transformation)
transformed_features = np.asarray(transformed_features)
transformed_features = np.reshape(transformed_features,(1,-1))
data_point_for_copula_1 = pd.DataFrame(transformed_features)
copula_1_output = dist_1.cdf(data_point_for_copula_1)

data_point_for_copula_2 = np.asarray([copula_1_output,current_feature[-1,:][0]])
data_point_for_copula_2 = np.resahpe(data_point_for_copula_2,(1,1))
data_point_for_copula_2 = pd.DataFrame(data_point_for_copula_2)
copula_2_output = dist_1.cdf(data_point_for_copula_2)

In [ ]:
free_parameters = \
{'cov_para_1': 0.03486785888671873, 'cov_para_2': 0.11317462921142585, 'cov_para_3': 0.1690827012062074, 'cov_para_4': 0.3950744748115539, 'beta_1a': 9.527766108512878, 'beta_1b': 8.363484144210815, 'beta_2a': 17.49446749687195, 'beta_2b': 14.08113718032837, 'beta_3a': 0.5255872756242752, 'beta_3b': 18.776148557662964, 'beta_4a': 16.938138008117676, 'beta_4b': 10.301854610443115, 'beta_5a': 12.012619972229004, 'beta_5b': 9.151394367218018, 'lambda_expon_1': 0.08449697780609132, 'lambda_expon_2': 0.07192293000221253, 'lambda_expon_3': 0.09399740934371949}


In [ ]:
lambda_expon_1 = free_parameters['lambda_expon_1']
lambda_expon_2 = free_parameters['lambda_expon_2']
lambda_expon_3 = free_parameters['lambda_expon_3']
lambda_expons = [lambda_expon_1,lambda_expon_2,lambda_expon_3]

In [ ]:
import utils
from scipy.stats import expon
import numpy as np
import pandas as pd

In [ ]:
import trading_vix_objective_2
locol_env = trading_vix_objective_2.trading_vix()

In [ ]:
current_feature = locol_env.reset()

In [ ]:
cov_input = [free_parameters['cov_para_1'],free_parameters['cov_para_2'],free_parameters['cov_para_3']]
marginal_input = [free_parameters['beta_1a'],free_parameters['beta_1b'],
         free_parameters['beta_2a'],free_parameters['beta_2b'],
         free_parameters['beta_3a'],free_parameters['beta_3b']]
dist_1 = utils.construct_a_copula(cov_input,marginal_input)

cov_input = [free_parameters['cov_para_4']]
marginal_input = [free_parameters['beta_4a'],free_parameters['beta_4b'],
         free_parameters['beta_5a'],free_parameters['beta_5b']]
dist_2 = utils.construct_a_copula(cov_input,marginal_input)

In [ ]:
transformed_features = []
for feature_index in range(len(lambda_expons)):
    transformation = expon.cdf(current_feature[feature_index,0],scale = 1.0/lambda_expons[feature_index])
    transformed_features.append(transformation)
transformed_features = np.asarray(transformed_features)
transformed_features = np.reshape(transformed_features,(1,-1))
data_point_for_copula_1 = pd.DataFrame(transformed_features)
copula_1_output = dist_1.cdf(data_point_for_copula_1)

In [ ]:
data_point_for_copula_1

In [ ]:
copula_1_output

In [ ]:
data_point_for_copula_2 = np.asarray([copula_1_output,current_feature[-1,:][0]])
data_point_for_copula_2 = np.asarray([0.5,0.3])
data_point_for_copula_2 = np.reshape(data_point_for_copula_2,(1,-1))
data_point_for_copula_2 = pd.DataFrame(data_point_for_copula_2)
copula_2_output = dist_2.cdf(data_point_for_copula_2)

In [ ]:
data_point_for_copula_2

In [ ]:
copula_2_output

In [ ]:
from sympy.solvers import solve
from sympy import Symbol

value_in_stock = 9000
cash = 0.01
action = 0.95
x = Symbol('x')
r = solve((value_in_stock-x)/(value_in_stock-x+cash) - action,x)

In [ ]:
r

In [1]:
free_parameters = \
{'cov_para_1': 0.7077383995056153, 'cov_para_2': 0.04921081066131594, 'cov_para_3': -0.6062130600214004, 'cov_para_4': -0.31497657895088194, 'cov_para_5': 0.3259180068969726, 'cov_para_6': -0.11180943846702573, 'beta_1a': 19.896345138549805, 'beta_1b': 1.0905852913856506, 'beta_2a': 3.6441731452941895, 'beta_2b': 11.095696687698364, 'beta_3a': 9.475186467170715, 'beta_3b': 12.109053134918213, 'beta_4a': 5.415239930152893, 'beta_4b': 1.1566774547100067, 'lambda_expon_1': 0.029026871383190156, 'lambda_expon_2': 0.022337756991386414, 'lambda_expon_3': 0.04982164895534516}

In [2]:
from copulas.multivariate import Multivariate
import numpy as np
import pandas as pd

In [3]:
free_para1 = free_parameters['cov_para_1']
free_para2 = free_parameters['cov_para_2']
free_para3 = free_parameters['cov_para_3']
free_para4 = free_parameters['cov_para_4']
free_para5 = free_parameters['cov_para_5']
free_para6 = free_parameters['cov_para_6']

diag_1 = 1
diag_2 = np.sqrt(1-free_para1**2)
if (1-free_para2**2-free_para3**2)<0:
    diag_3 = np.sqrt(np.abs(1-free_para2**2-free_para3**2))
else:
    diag_3 = np.sqrt(1-free_para2**2-free_para3**2)
if (1-free_para4**2-free_para5**2-free_para6**2)<0:
    diag_4 = np.sqrt(np.abs(1-free_para4**2-free_para5**2-free_para6**2))
else:
    diag_4 = np.sqrt(1-free_para4**2-free_para5**2-free_para6**2)


lower_triangular_matrix = np.asarray([[diag_1,0,0,0],
                        [free_para1,diag_2,0,0],
                        [free_para2,free_para3,diag_3,0],
                        [free_para4,free_para5,free_para6,diag_4]])


cov_matrix = lower_triangular_matrix@lower_triangular_matrix.transpose()


#now, define the marginal distribution of the gaussian copula
univerates = [{'loc': 0,
'scale': 1,
'a': free_parameters['beta_1a'],
'b': free_parameters['beta_1b'],
'type': 'copulas.univariate.beta.BetaUnivariate'},
{'loc': 0,
'scale': 1,
'a': free_parameters['beta_2a'],
'b': free_parameters['beta_2b'],
'type': 'copulas.univariate.beta.BetaUnivariate'},
{'loc': 0,
'scale': 1,
'a': free_parameters['beta_3a'],
'b': free_parameters['beta_3b'],
'type': 'copulas.univariate.beta.BetaUnivariate'},
{'loc': 0,
'scale': 1,
'a': free_parameters['beta_4a'],
'b': free_parameters['beta_4b'],
'type': 'copulas.univariate.beta.BetaUnivariate'}]


#now, we construct the gaussian copula
copula_parameters = {}
copula_parameters['covariance'] = cov_matrix
copula_parameters['univariates'] = univerates
copula_parameters['type'] = 'copulas.multivariate.gaussian.GaussianMultivariate'
copula_parameters['columns'] = [0,1,2,3]


new_dist = Multivariate.from_dict(copula_parameters)

In [4]:
import trading_vix
locol_env = trading_vix.trading_vix()
current_feature = locol_env.reset()

In [5]:
current_feature

array([[ 9.],
       [25.],
       [64.],
       [ 0.]])

In [6]:
type(current_feature)

numpy.ndarray

In [7]:
current_feature.dtype

dtype('float64')

In [8]:
current_feature,reward = locol_env.step(0.4)

<class 'sympy.core.numbers.Float'>


In [9]:
current_feature

array([[10.0],
       [26.0],
       [65.0],
       [0.398478084374039]], dtype=object)

In [ ]:
current_feature.dtype

In [ ]:
locol_env.index_feature_dataframe.iloc[locol_env.current_time_index][0]

In [ ]:
current_feature[-1,:]

In [ ]:
transformed_features = []
for feature_index in range(len(lambda_expons)):
    transformation = expon.cdf(current_feature[feature_index,0],scale = 1.0/lambda_expons[feature_index])
    transformed_features.append(transformation)
transformed_features = np.asarray(transformed_features)
transformed_features = np.reshape(transformed_features,(1,-1))
holding_position = expit(current_feature[-1,:][0])
holding_position = np.reshape(holding_position,(1,1))
data_point_for_df = np.concatenate((transformed_features,holding_position),axis = 1)

In [ ]:
type(current_feature[-1,:][0])

In [ ]:
current_feature[-1,:][0]

In [ ]:
current_feature[-1,:]

In [ ]:
type(expit(3))

In [ ]:
current_feature[-1,:]

In [ ]:
holding_position